In [ ]:
import lizard
import pandas as pd
import matplotlib

In [ ]:
# lizardのテスト
i = lizard.analyze_file("test.java")
print(i.__dict__)
for func in i.function_list:
  print( func.__dict__["name"])

In [ ]:

diff_df = pd.read_pickle("../1.scraping/scraped_files/chromium/diff.pkl")

In [ ]:
diff_df.info()

In [ ]:
len(diff_df.content[1])

In [ ]:
diff_df["has_diff"] = diff_df["content"].apply(lambda x: True if len(x) != 1 else False)

In [ ]:
diff_df["has_diff"].value_counts()

In [ ]:
diff_df_has_diff = diff_df[diff_df["has_diff"]]

In [ ]:
diff_df_has_diff["is_java"] = diff_df_has_diff["meta_a.name"].str.contains(".java")

In [ ]:
diff_df_has_diff["is_java"].value_counts()

In [ ]:
def separate_before_after(contents,comment_id):
  before = ""
  after = ""
  for content in contents:
    if "a" in content:
      after += content["a"][0] + "\n"
    elif "b" in content:
      before += content["b"][0]+ "\n"
    elif "ab" in content:
      after += content["ab"][0]+ "\n"
      before += content["ab"][0]+ "\n"
  with open(comment_id+"_before","w", encoding="utf-8", newline="\n") as f:
    f.write(before)
  with open(comment_id+"_after","w", encoding="utf-8", newline="\n") as f:
    f.write(after)
separate_before_after(diff_df_has_diff.iloc[0]["content"],diff_df_has_diff.iloc[0]["comments_id"])

In [ ]:
diff_df_has_diff.iloc[0]["content"]

In [ ]:
df = pd.read_pickle("../1.scraping/scraped_files/opendev/diff.pkl")

In [ ]:
df["meta_a.name"].unique()

In [ ]:
df["is_java"] = df["meta_a.name"].str.contains(".java")

In [ ]:
df["is_java"].value_counts()

In [ ]:
diff_df = pd.read_pickle("../1.scraping/scraped_files/chromium/diff.pkl")


In [ ]:
diff_df["has_diff"] = diff_df["content"].apply(lambda x: len(x) != 1)
diff_df["is_java"]  = diff_df["meta_a.name"].str.contains(".java")
df_temp = diff_df[diff_df["is_java"] & diff_df["has_diff"] ].loc[:,["comments_id","content"]].values

In [ ]:
def separate_before_after(comment_id,contents):
  before = list()
  after = list()
  for content in contents:
    if "a" in content:
      after.extend(content["a"])
    elif "b" in content:
      before.extend(content["b"])
    elif "ab" in content:
      after.extend(content["ab"])
      before.extend(content["ab"])
  with open("./files/"+comment_id+"_before.java","w", encoding="utf-8") as f:
    f.write("\n".join(before))
  with open("./files/"+comment_id+"_after.java","w", encoding="utf-8") as f:
    f.write("\n".join(after))


In [ ]:
[*map(lambda x: separate_before_after(x[0],x[1]), df_temp )]

In [ ]:
df_temp = diff_df[diff_df["is_java"] & diff_df["has_diff"] ]

In [ ]:
df_temp.info()

In [ ]:
comments_df = pd.read_pickle("../1.scraping/scraped_files/chromium/comments.pkl")
comments_df.info()

In [ ]:
# merge
comments_df = comments_df.rename(columns={'id': 'comments_id'})
merge_df = pd.merge(comments_df, df_temp)

In [ ]:
merge_df= merge_df.loc[:,["comments_id","line","message","range.start_line","range.end_line"]]

In [ ]:
merge_df["filepath_before"] = merge_df["comments_id"].apply(lambda x: x+"_before.java")
merge_df["filepath_after"] = merge_df["comments_id"].apply(lambda x: x+"_after.java")
merge_df["is_gerrit"] = True
merge_df["repository_name"] = "chromium"




In [ ]:
df = pd.read_pickle("./review_df.pkl")

In [ ]:
df